# Getting started with ONNX IR 🌱
The ONNX IR ships with the ONNX Script package and is available as `onnxscript.ir`.
To create an IR object from ONNX file, load it as `ModelProto` and call
`ir.from_proto()` or `ir.serde.deserialize_model`:

In [1]:
import pathlib

import onnx

from onnxscript import ir

# Load the model as onnx.ModelProto
model_proto = onnx.load(
    pathlib.Path(ir.__file__).parent.parent.parent
    / "testdata"
    / "dort_models"
    / "llama_forward.onnx"
)

# Create an IR object from the model
model = ir.serde.deserialize_model(model_proto)

Now we can explore the IR object

In [2]:
print(f"The main graph has {len(model.graph)} nodes.")

The main graph has 279 nodes.


All inputs

In [3]:
print(model.graph.inputs)

[Input('primals_8', type=Tensor(FLOAT), shape=[2,1,1024,1024], producer=None, index=None), Input('primals_1', type=Tensor(FLOAT), shape=[16,16], producer=None, index=None), Input('primals_6', type=Tensor(FLOAT), shape=[2,1024,16], producer=None, index=None), Input('primals_4', type=Tensor(FLOAT), shape=[16,16], producer=None, index=None), Input('primals_2', type=Tensor(FLOAT), shape=[16,16], producer=None, index=None), Input('primals_3', type=Tensor(FLOAT), shape=[16,16], producer=None, index=None), Input('primals_5', type=Tensor(FLOAT), shape=[4], producer=None, index=None), Input('primals_7', type=Tensor(INT64), shape=[1,1024], producer=None, index=None)]


All outputs

In [4]:
print(model.graph.outputs)

[Value('view', type=Tensor(FLOAT), shape=[2048,16], producer=True, index=0), Value('t_6', type=Tensor(FLOAT), shape=[16,16], producer=True, index=0), Value('transpose_8', type=Tensor(FLOAT), shape=[4,8,1024], producer=True, index=0), Value('cat', type=Tensor(FLOAT), shape=[1,1024,8], producer=True, index=0), Value('transpose_9', type=Tensor(FLOAT), shape=[4,8,1024], producer=True, index=0), Value('transpose_10', type=Tensor(FLOAT), shape=[4,1024,8], producer=True, index=0), Value('detach_3', type=Tensor(FLOAT), shape=[2,2,1024,1024], producer=True, index=0), Value('transpose_7', type=Tensor(FLOAT), shape=[4,1024,1024], producer=True, index=0), Value('view_19', type=Tensor(FLOAT), shape=[2048,16], producer=True, index=0), Value('view_20', type=Tensor(FLOAT), shape=[2,1024,16], producer=True, index=0)]


Nodes that uses the first input

In [5]:
print(list(model.graph.inputs[0].uses()))

[(Node(name='Slice_83', domain='', op_type='Slice', inputs=(Input('primals_8', type=Tensor(FLOAT), shape=[2,1,1024,1024], producer=None, index=None), Value('_val_11', type=None, shape=None, producer=True, index=0), Value('_val_15', type=None, shape=None, producer=True, index=0), Value('_val_19', type=None, shape=None, producer=True, index=0), Value('_val_23', type=None, shape=None, producer=True, index=0)), attributes=OrderedDict(), overload='', outputs=(Value('slice_8', type=Tensor(FLOAT), shape=[2,1,1024,1024], producer=True, index=0),), version=None, doc_string=''), 0)]


The node that produces the last output (as the i-th output)

In [6]:
print(model.graph.outputs[-1].producer())
print(model.graph.outputs[-1].index())

%"view_20"<FLOAT,[2,1024,16]> ⬅️ pkg.onnxscript.torch_lib::aten_view(%"mm_3", %"_val_285")
0


Examine a Function

In [7]:
print(model.functions[("pkg.onnxscript.torch_lib", "aten_view", "")])

<
    opset_imports={'': 18},
>
def pkg.onnxscript.torch_lib::aten_view(
    inputs=(
        %"self"<?,?>,
        %"size"<?,?>
    ),
    outputs=(
        %"return_val"<?,?>
    ),
) {
    0 |  # n0
         %"size_0"<?,?> ⬅️ ::Cast(%"size") {to=7}
    1 |  # n1
         %"return_val"<?,?> ⬅️ ::Reshape(%"self", %"size_0")
    return %"return_val"<?,?>
}


Print the graph

In [8]:
model.graph.display(
    page=False
)  # Set page=True to use a pager in the terminal so long outputs are scrollable

graph(
    name=main_graph,
    inputs=(
        %"primals_8"<FLOAT,[2,1,1024,1024]>,
        %"primals_1"<FLOAT,[16,16]>,
        %"primals_6"<FLOAT,[2,1024,16]>,
        %"primals_4"<FLOAT,[16,16]>,
        %"primals_2"<FLOAT,[16,16]>,
        %"primals_3"<FLOAT,[16,16]>,
        %"primals_5"<FLOAT,[4]>,
        %"primals_7"<INT64,[1,1024]>
    ),
    outputs=(
        %"view"<FLOAT,[2048,16]>,
        %"t_6"<FLOAT,[16,16]>,
        %"transpose_8"<FLOAT,[4,8,1024]>,
        %"cat"<FLOAT,[1,1024,8]>,
        %"transpose_9"<FLOAT,[4,8,1024]>,
        %"transpose_10"<FLOAT,[4,1024,8]>,
        %"detach_3"<FLOAT,[2,2,1024,1024]>,
        %"transpose_7"<FLOAT,[4,1024,1024]>,
        %"view_19"<FLOAT,[2048,16]>,
        %"view_20"<FLOAT,[2,1024,16]>
    ),
) {
      0 |  # Constant_67
           %"_val_8"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
      1 |  # Cast_68
           %"_val_9"<?,?> ⬅️ ::Cast(%"_val_8") {to=7}
      2 |  # Constant_69
           %"_val_10"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
      3 |  # Reshape_70
           %"_val_11"<?,?> ⬅️ ::Reshape(%"_val_9", %"_val_10") {allowzero=0}
      4 |  # Constant_71
           %"_val_12"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
      5 |  # Cast_72
           %"_val_13"<?,?> ⬅️ ::Cast(%"_val_12") {to=7}
      6 |  # Constant_73
           %"_val_14"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
      7 |  # Reshape_74
           %"_val_15"<?,?> ⬅️ ::Reshape(%"_val_13", %"_val_14") {allowzero=0}
      8 |  # Constant_75
           %"_val_16"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
      9 |  # Cast_76
           %"_val_17"<?,?> ⬅️ ::Cast(%"_val_16") {to=7}
     10 |  # Constant_77
           %"_val_18"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
     11 |  # Reshape_78
           %"_val_19"<?,?> ⬅️ ::Reshape(%"_val_17", %"_val_18") {allowzero=0}
     12 |  # Constant_79
           %"_val_20"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
     13 |  # Cast_80
           %"_val_21"<?,?> ⬅️ ::Cast(%"_val_20") {to=7}
     14 |  # Constant_81
           %"_val_22"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
     15 |  # Reshape_82
           %"_val_23"<?,?> ⬅️ ::Reshape(%"_val_21", %"_val_22") {allowzero=0}
     16 |  # Slice_83
           %"slice_8"<FLOAT,[2,1,1024,1024]> ⬅️ ::Slice(%"primals_8", %"_val_11", %"_val_15", %"_val_19", 
%"_val_23")
     17 |  # aten_t_84
           %"t"<FLOAT,[16,16]> ⬅️ pkg.onnxscript.torch_lib::aten_t(%"primals_1")
     18 |  # Constant_85
           %"_val_26"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[2]>(name='')}
     19 |  # aten_view_86
           %"view"<FLOAT,[2048,16]> ⬅️ pkg.onnxscript.torch_lib::aten_view(%"primals_6", %"_val_26")
     20 |  # aten_t_87
           %"t_3"<FLOAT,[16,16]> ⬅️ pkg.onnxscript.torch_lib::aten_t(%"primals_4")
     21 |  # aten_t_88
           %"t_1"<FLOAT,[16,16]> ⬅️ pkg.onnxscript.torch_lib::aten_t(%"primals_2")
     22 |  # aten_t_89
           %"t_2"<FLOAT,[16,16]> ⬅️ pkg.onnxscript.torch_lib::aten_t(%"primals_3")
     23 |  # aten_unsqueeze_90
           %"unsqueeze"<FLOAT,[1,4]> ⬅️ pkg.onnxscript.torch_lib::aten_unsqueeze(%"primals_5") {dim=0}
     24 |  # Constant_91
           %"_val_32"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
     25 |  # Cast_92
           %"_val_33"<?,?> ⬅️ ::Cast(%"_val_32") {to=7}
     26 |  # Constant_93
           %"_val_34"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
     27 |  # Reshape_94
           %"_val_35"<?,?> ⬅️ ::Reshape(%"_val_33", %"_val_34") {allowzero=0}
     28 |  # Constant_95
           %"_val_36"<?,?> ⬅️ ::Constant() {value=TensorProtoTensor<INT64,[]>(name='')}
     29 |  # Cast_96
           %"_val_37"<?,?> ⬅️ ::Cast(%"_val_36") {to=7}
     30 |  # Constant_97
           %"_val_38"<?,?> ⬅️ ::Constant() {value_ints=[-1]}
     31 |  # Reshape_98
           %"_val_39"<?,?> ⬅️ ::Reshape(%"_val_37", %"_val_38") {allowzero=0}
     32 |  # Constant_99
      

Convert from the IR object back to ModelProto

In [9]:
model_proto_back = ir.serde.serialize_model(model)

## Next steps

Read the introductions for a more detailed introduction of the IR
(Documentation in progress 🚧)